In [1]:
%load_ext autoreload
%autoreload 2

from sci.metrics import rmse, rmse_score, neg_rmse_score
import sci.plots as scip
import sci.learn as scil

# %matplotlib inline
# import matplotlib
# matplotlib.rcParams['figure.figsize'] = (10, 6)

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.5f}'.format(x)) #Limiting floats output to 3 decimal points
pd.options.display.max_rows = 1000

# import seaborn as sns
# import matplotlib.pyplot as plt

random_state = 7

## Load Data

In [3]:
data = scil.load_data("processed_my2", random_state=random_state)


(891, 16) (891,)
train: (712, 16)
test: (179, 16)


## Setup

In [4]:
# from mlxtend.evaluate import BootstrapOutOfBag, RandomHoldoutSplit
from sklearn.metrics import *

metric = accuracy_score
scoring = 'accuracy'

# scil.metric_global = metric
# scil.scoring_global = scoring # make_scorer(scil.metric_global, needs_proba=True)

# format
# cv_{total_splits}_{train/test-fold}
# cv_1_10 = RandomHoldoutSplit(valid_size=0.1, stratify=True, random_seed=None)
cv_5_5 = 5
cv_10_10 = 10
# cv_1000_10 = RepeatedStratifiedKFold(n_splits=10, n_repeats=100)
# cv_1000_20 = RepeatedStratifiedKFold(n_splits=20, n_repeats=50)

# cv_single = cv_1_10
# cv_fast = cv_10_10
# cv_slow = cv_1000_20

def drop_str_features(df):
    return df.drop([c for c in df.columns if df[c].dtype == "object"], axis=1)


## Auto

In [5]:
X_train = data.X_train
y_train = data.y_train
X_test = data.X_test
y_test = data.y_test
X = data.X
y = data.y
X_score = data.X_score

X = drop_str_features(X)
X_score = drop_str_features(X_score)
X_train = drop_str_features(X_train)
X_test = drop_str_features(X_test)

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [7]:
from hpsklearn import HyperoptEstimator
import hpsklearn

In [17]:
estim = HyperoptEstimator(max_evals=10)
# estim.fit(X_train.values, y_train.values)

In [23]:
from sklearn.base import clone
clone(estim)

AttributeError: 'NoneType' object has no attribute 'randint'

In [12]:
y_pred = estim.predict(X_test)
accuracy_score(y_test, y_pred)

0.8044692737430168

In [27]:
from sklearn.model_selection import check_cv
from sklearn.metrics import get_scorer

def cross_val_score(est, X, y, scoring, cv):
    cv = check_cv(cv)
    scorer = get_scorer(scoring)
    scores = []
    for train, test in cv.split(X, y):
        est.fit(X[train], y[train])
        scores.append(scorer(est, X[test], y[test]))
    return scores
    
cross_val_score(estim, X.values, y.values, scoring="accuracy", cv=5)

100%|██████████| 1/1 [00:00<00:00,  7.07it/s, best loss: 0.08391608391608396]


[0.7932960893854749,
 0.8370786516853933,
 0.898876404494382,
 0.8651685393258427,
 0.8146067415730337]

In [29]:
a = [0.7932960893854749,
 0.8370786516853933,
 0.898876404494382,
 0.8651685393258427,
 0.8146067415730337]
import numpy as np
np.mean([0.7932960893854749,
 0.8370786516853933,
 0.898876404494382,
 0.8651685393258427,
 0.8146067415730337])

0.8418052852928254

In [31]:
estim = HyperoptEstimator(max_evals=30)
estim.fit(X.values, y.values)

100%|██████████| 1/1 [00:00<00:00, 12.80it/s, best loss: 0.17318435754189943]
